In [1]:
import random
from string import ascii_uppercase, ascii_letters
from pprint import pprint
from copy import deepcopy
import dill

In [2]:
## initial data needed to generate setup of rotors through Bombe Scrambler
# entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
# raw_rotors = {'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}
# forward_rotors = {k:[ascii_uppercase.index(c) for c in raw_rotors[k]] for k in raw_rotors.keys()}
# rev_rotors = {}
# for r in raw_rotors.keys():
#     working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
#     rev_rotors[r] = [working[k] for k in sorted(working.keys())]
# reflectors = {'Brf': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 
#               'Crf': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}

entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
raw_rotors = {'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}
notches = {'I':'Q','II':'E','III':'V','IV':'J','V':'Z'}
## forward rotors is the forward in:out pairings of each rotor as the character index of the A-Z ascii alphabet stored in 'entry'
forward_rotors = {k:[ascii_uppercase.index(c) for c in raw_rotors[k]] for k in raw_rotors.keys()}

## couldn't see a straightforward list comprehension for the reverse - the in:out pairing for when the current flows
## back from the reflector to the final output. Hopefully this two-step for loop isn't too slow
rev_rotors = {}
for r in raw_rotors.keys():
    working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
    rev_rotors[r] = [working[k] for k in sorted(working.keys())]

# for k,v in rev_rotors.items():
#     print(k,':',v)

## in:out pairings for reflectors are hardcoded in, were originally generated by code
reflectors = {'B': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 'C': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}
# print(reflectors)

In [3]:
class ScramblerBombe:
    
    def __init__(self,left_rotor,middle_rotor,right_rotor,reflector,menu_link='ZZZ',conx_in={},conx_out={}):
        """rotors must be strings referring to either ['I','II','III','IV','V']
        reflector must be string, one of either ['B','C']"""
        
        self.right_rotor = right_rotor
        self.middle_rotor = middle_rotor
        self.left_rotor = left_rotor
        self.reflector = reflectors[reflector]
        self.menu_link = menu_link
        self.middle_notch = entry.index(notches[self.middle_rotor])   ## point if right rotor reaches will trigger middle rotor to step
        self.left_notch = entry.index(notches[self.left_rotor])  ## point if middle rotor reaches will trigger left rotor to step
        self.pos_left_rotor, self.pos_mid_rotor, self.pos_rgt_rotor = (ascii_uppercase.index(m) for m in menu_link.upper())
        self.in_status = {char:0 for char in entry}
        self.out_status = {char:0 for char in entry}
        self.conx_in = conx_in
        self.conx_out = conx_out
        
    def once_thru_scramble(self,start_character, direction, first_rotor, pos1, second_rotor, pos2, 
                       third_rotor, pos3):
        """ start_character must be single ASCII character A-Z
        direction is either 'forward' or 'back' """
        if direction == 'forward':
            usedict = {k:v for k,v in forward_rotors.items()}
        elif direction == 'back':
            usedict = {k:v for k,v in rev_rotors.items()}
        else:
            print('only forward or back for direction')
            return 'wtf'

        start_character = start_character.upper()
        entry_pos = entry.index(start_character)
        fst_pos_modifier = (26 + pos1 - 0)%26
        fst_in = (entry_pos + fst_pos_modifier)%26
        fst_out = usedict[first_rotor][fst_in]
        ch1o = entry[fst_out]

        scd_pos_modifier = (26 + pos2 - pos1)%26
        scd_in = (fst_out + scd_pos_modifier)%26
        ch2i = entry[scd_in]
        scd_out = usedict[second_rotor][scd_in]
        ch2o = entry[scd_out]

        thd_pos_modifier = (26 + pos3 - pos2)%26
        thd_in = (scd_out + thd_pos_modifier)%26
        ch3i = entry[thd_in]
        thd_out = usedict[third_rotor][thd_in]
        ch3o = entry[thd_out]

#         if direction == 'forward':
#             print(f"{start_character} -> (RR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (LR in) {ch3i} -> (LR out) {ch3o}")
#         elif direction == 'back':
#             print(f"{start_character} -> (LR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (RR in) {ch3i} -> (RR out) {ch3o}")

        return ch3o
    
    def full_scramble(self,in_ch):
        in_ch = in_ch.upper()
        left_rotor = self.left_rotor
        middle_rotor = self.middle_rotor
        right_rotor = self.right_rotor
        rflector = self.reflector
        ## first run right to left through scrambler
        forward_run = self.once_thru_scramble(in_ch, direction='forward', first_rotor=right_rotor, pos1=self.pos_rgt_rotor, 
                                              second_rotor=middle_rotor, pos2=self.pos_mid_rotor, 
                                              third_rotor=left_rotor, pos3=self.pos_left_rotor)

        ## reflector back around for return
        rfi_pos_mod = (26 + 0 - self.pos_left_rotor)%26    ## the '0' is there to matching formatting of other position modifiers - reflector is not moved so it will always be 0
        rf_in = (entry.index(forward_run) + rfi_pos_mod)%26
        chri = entry[rf_in]
        mirrored = rflector[chri]

#         print(f"{forward_run} -> {chri} (into reflector) -> {mirrored} (reflected out)")

        ## second run back left to right thru scrambler
        back_run = self.once_thru_scramble(mirrored, direction='back', first_rotor=left_rotor, pos1=self.pos_left_rotor, 
                                      second_rotor=middle_rotor, pos2=self.pos_mid_rotor, third_rotor=right_rotor, pos3=self.pos_rgt_rotor)

        bk_out = entry.index(back_run)
        bko_pos_mod = (26 + 0 - self.pos_rgt_rotor)%26  ## as above, '0' just reflects that the entry interface doesn't move
        bk_final = (bk_out + bko_pos_mod)%26
        final = entry[bk_final]
#         print('RR back out:  ', back_run, '-->', final)
#         print(in_ch,"-->",final)
        return final
    
    def rotor_step(self,rotor_position):
        """"""
        if rotor_position == 25:
            rotor_position = 0
        else:
            rotor_position += 1
        return rotor_position
    
    def step_enigma(self):
        """Just acts on itself, steps the rotors"""
        
#         print(f"Lpos={self.pos_left_rotor}, Mpos={self.pos_mid_rotor}, Rpos={self.pos_rgt_rotor}")
        
        if self.pos_rgt_rotor == self.middle_notch and self.pos_mid_rotor == self.left_notch:
            self.pos_rgt_rotor = self.rotor_step(self.pos_rgt_rotor)
            self.pos_mid_rotor = self.rotor_step(self.pos_mid_rotor)
            self.pos_left_rotor = self.rotor_step(self.pos_left_rotor)
            print('--- left & middle rotor step')
        elif self.pos_rgt_rotor == self.middle_notch:
            self.pos_rgt_rotor = self.rotor_step(self.pos_rgt_rotor)
            self.pos_mid_rotor = self.rotor_step(self.pos_mid_rotor)
            print('--- middle rotor step ---')
        else:
            self.pos_rgt_rotor = self.rotor_step(self.pos_rgt_rotor)
            
        print(f"Lpos={self.pos_left_rotor}, Mpos={self.pos_mid_rotor}, Rpos={self.pos_rgt_rotor}")

    def update(self):
        """idea here is that the scrambler will check each of the 26 connections to see if they
        are live, and if so pass it through itself to live up the connection on the other side of 
        the scrambler"""
        
        for char, status in self.in_status.items():
            if status == 0:
                pass
            else:
                self.out_status[self.full_scramble(char)] = 1
                
        for char, status in self.out_status.items():
            if status == 0:
                pass
            else:
                self.in_status[self.full_scramble(char)] = 1
            


            

In [4]:
eb1 = ScramblerBombe(left_rotor='II',middle_rotor='V',right_rotor='III',reflector='C',menu_link='BCD')

In [5]:
# eb1.in_status

In [6]:
eb1.in_status['E'] = 1
eb1.in_status['H'] = 1
eb1.in_status['P'] = 1
eb1.in_status['X'] = 1
eb1.in_status

{'A': 0,
 'B': 0,
 'C': 0,
 'D': 0,
 'E': 1,
 'F': 0,
 'G': 0,
 'H': 1,
 'I': 0,
 'J': 0,
 'K': 0,
 'L': 0,
 'M': 0,
 'N': 0,
 'O': 0,
 'P': 1,
 'Q': 0,
 'R': 0,
 'S': 0,
 'T': 0,
 'U': 0,
 'V': 0,
 'W': 0,
 'X': 1,
 'Y': 0,
 'Z': 0}

In [7]:
# eb1.out_status

In [8]:
eb1.update()
eb1.out_status

{'A': 0,
 'B': 0,
 'C': 0,
 'D': 0,
 'E': 1,
 'F': 0,
 'G': 1,
 'H': 1,
 'I': 0,
 'J': 0,
 'K': 0,
 'L': 0,
 'M': 0,
 'N': 0,
 'O': 0,
 'P': 0,
 'Q': 0,
 'R': 0,
 'S': 0,
 'T': 0,
 'U': 0,
 'V': 0,
 'W': 1,
 'X': 0,
 'Y': 0,
 'Z': 0}

In [9]:
%%time
for i in range(26**3):
    eb1.full_scramble('Q')
    eb1.step_enigma()

Lpos=1, Mpos=2, Rpos=4
Lpos=1, Mpos=2, Rpos=5
Lpos=1, Mpos=2, Rpos=6
Lpos=1, Mpos=2, Rpos=7
Lpos=1, Mpos=2, Rpos=8
Lpos=1, Mpos=2, Rpos=9
Lpos=1, Mpos=2, Rpos=10
Lpos=1, Mpos=2, Rpos=11
Lpos=1, Mpos=2, Rpos=12
Lpos=1, Mpos=2, Rpos=13
Lpos=1, Mpos=2, Rpos=14
Lpos=1, Mpos=2, Rpos=15
Lpos=1, Mpos=2, Rpos=16
Lpos=1, Mpos=2, Rpos=17
Lpos=1, Mpos=2, Rpos=18
Lpos=1, Mpos=2, Rpos=19
Lpos=1, Mpos=2, Rpos=20
Lpos=1, Mpos=2, Rpos=21
Lpos=1, Mpos=2, Rpos=22
Lpos=1, Mpos=2, Rpos=23
Lpos=1, Mpos=2, Rpos=24
Lpos=1, Mpos=2, Rpos=25
--- middle rotor step ---
Lpos=1, Mpos=3, Rpos=0
Lpos=1, Mpos=3, Rpos=1
Lpos=1, Mpos=3, Rpos=2
Lpos=1, Mpos=3, Rpos=3
Lpos=1, Mpos=3, Rpos=4
Lpos=1, Mpos=3, Rpos=5
Lpos=1, Mpos=3, Rpos=6
Lpos=1, Mpos=3, Rpos=7
Lpos=1, Mpos=3, Rpos=8
Lpos=1, Mpos=3, Rpos=9
Lpos=1, Mpos=3, Rpos=10
Lpos=1, Mpos=3, Rpos=11
Lpos=1, Mpos=3, Rpos=12
Lpos=1, Mpos=3, Rpos=13
Lpos=1, Mpos=3, Rpos=14
Lpos=1, Mpos=3, Rpos=15
Lpos=1, Mpos=3, Rpos=16
Lpos=1, Mpos=3, Rpos=17
Lpos=1, Mpos=3, Rpos=18
Lpos=1

Lpos=4, Mpos=21, Rpos=23
Lpos=4, Mpos=21, Rpos=24
Lpos=4, Mpos=21, Rpos=25
--- middle rotor step ---
Lpos=4, Mpos=22, Rpos=0
Lpos=4, Mpos=22, Rpos=1
Lpos=4, Mpos=22, Rpos=2
Lpos=4, Mpos=22, Rpos=3
Lpos=4, Mpos=22, Rpos=4
Lpos=4, Mpos=22, Rpos=5
Lpos=4, Mpos=22, Rpos=6
Lpos=4, Mpos=22, Rpos=7
Lpos=4, Mpos=22, Rpos=8
Lpos=4, Mpos=22, Rpos=9
Lpos=4, Mpos=22, Rpos=10
Lpos=4, Mpos=22, Rpos=11
Lpos=4, Mpos=22, Rpos=12
Lpos=4, Mpos=22, Rpos=13
Lpos=4, Mpos=22, Rpos=14
Lpos=4, Mpos=22, Rpos=15
Lpos=4, Mpos=22, Rpos=16
Lpos=4, Mpos=22, Rpos=17
Lpos=4, Mpos=22, Rpos=18
Lpos=4, Mpos=22, Rpos=19
Lpos=4, Mpos=22, Rpos=20
Lpos=4, Mpos=22, Rpos=21
Lpos=4, Mpos=22, Rpos=22
Lpos=4, Mpos=22, Rpos=23
Lpos=4, Mpos=22, Rpos=24
Lpos=4, Mpos=22, Rpos=25
--- middle rotor step ---
Lpos=4, Mpos=23, Rpos=0
Lpos=4, Mpos=23, Rpos=1
Lpos=4, Mpos=23, Rpos=2
Lpos=4, Mpos=23, Rpos=3
Lpos=4, Mpos=23, Rpos=4
Lpos=4, Mpos=23, Rpos=5
Lpos=4, Mpos=23, Rpos=6
Lpos=4, Mpos=23, Rpos=7
Lpos=4, Mpos=23, Rpos=8
Lpos=4, Mpos=23, 

Lpos=7, Mpos=17, Rpos=25
--- middle rotor step ---
Lpos=7, Mpos=18, Rpos=0
Lpos=7, Mpos=18, Rpos=1
Lpos=7, Mpos=18, Rpos=2
Lpos=7, Mpos=18, Rpos=3
Lpos=7, Mpos=18, Rpos=4
Lpos=7, Mpos=18, Rpos=5
Lpos=7, Mpos=18, Rpos=6
Lpos=7, Mpos=18, Rpos=7
Lpos=7, Mpos=18, Rpos=8
Lpos=7, Mpos=18, Rpos=9
Lpos=7, Mpos=18, Rpos=10
Lpos=7, Mpos=18, Rpos=11
Lpos=7, Mpos=18, Rpos=12
Lpos=7, Mpos=18, Rpos=13
Lpos=7, Mpos=18, Rpos=14
Lpos=7, Mpos=18, Rpos=15
Lpos=7, Mpos=18, Rpos=16
Lpos=7, Mpos=18, Rpos=17
Lpos=7, Mpos=18, Rpos=18
Lpos=7, Mpos=18, Rpos=19
Lpos=7, Mpos=18, Rpos=20
Lpos=7, Mpos=18, Rpos=21
Lpos=7, Mpos=18, Rpos=22
Lpos=7, Mpos=18, Rpos=23
Lpos=7, Mpos=18, Rpos=24
Lpos=7, Mpos=18, Rpos=25
--- middle rotor step ---
Lpos=7, Mpos=19, Rpos=0
Lpos=7, Mpos=19, Rpos=1
Lpos=7, Mpos=19, Rpos=2
Lpos=7, Mpos=19, Rpos=3
Lpos=7, Mpos=19, Rpos=4
Lpos=7, Mpos=19, Rpos=5
Lpos=7, Mpos=19, Rpos=6
Lpos=7, Mpos=19, Rpos=7
Lpos=7, Mpos=19, Rpos=8
Lpos=7, Mpos=19, Rpos=9
Lpos=7, Mpos=19, Rpos=10
Lpos=7, Mpos=19, R

Lpos=10, Mpos=24, Rpos=11
Lpos=10, Mpos=24, Rpos=12
Lpos=10, Mpos=24, Rpos=13
Lpos=10, Mpos=24, Rpos=14
Lpos=10, Mpos=24, Rpos=15
Lpos=10, Mpos=24, Rpos=16
Lpos=10, Mpos=24, Rpos=17
Lpos=10, Mpos=24, Rpos=18
Lpos=10, Mpos=24, Rpos=19
Lpos=10, Mpos=24, Rpos=20
Lpos=10, Mpos=24, Rpos=21
Lpos=10, Mpos=24, Rpos=22
Lpos=10, Mpos=24, Rpos=23
Lpos=10, Mpos=24, Rpos=24
Lpos=10, Mpos=24, Rpos=25
--- middle rotor step ---
Lpos=10, Mpos=25, Rpos=0
Lpos=10, Mpos=25, Rpos=1
Lpos=10, Mpos=25, Rpos=2
Lpos=10, Mpos=25, Rpos=3
Lpos=10, Mpos=25, Rpos=4
Lpos=10, Mpos=25, Rpos=5
Lpos=10, Mpos=25, Rpos=6
Lpos=10, Mpos=25, Rpos=7
Lpos=10, Mpos=25, Rpos=8
Lpos=10, Mpos=25, Rpos=9
Lpos=10, Mpos=25, Rpos=10
Lpos=10, Mpos=25, Rpos=11
Lpos=10, Mpos=25, Rpos=12
Lpos=10, Mpos=25, Rpos=13
Lpos=10, Mpos=25, Rpos=14
Lpos=10, Mpos=25, Rpos=15
Lpos=10, Mpos=25, Rpos=16
Lpos=10, Mpos=25, Rpos=17
Lpos=10, Mpos=25, Rpos=18
Lpos=10, Mpos=25, Rpos=19
Lpos=10, Mpos=25, Rpos=20
Lpos=10, Mpos=25, Rpos=21
Lpos=10, Mpos=25, Rpos

Lpos=13, Mpos=10, Rpos=1
Lpos=13, Mpos=10, Rpos=2
Lpos=13, Mpos=10, Rpos=3
Lpos=13, Mpos=10, Rpos=4
Lpos=13, Mpos=10, Rpos=5
Lpos=13, Mpos=10, Rpos=6
Lpos=13, Mpos=10, Rpos=7
Lpos=13, Mpos=10, Rpos=8
Lpos=13, Mpos=10, Rpos=9
Lpos=13, Mpos=10, Rpos=10
Lpos=13, Mpos=10, Rpos=11
Lpos=13, Mpos=10, Rpos=12
Lpos=13, Mpos=10, Rpos=13
Lpos=13, Mpos=10, Rpos=14
Lpos=13, Mpos=10, Rpos=15
Lpos=13, Mpos=10, Rpos=16
Lpos=13, Mpos=10, Rpos=17
Lpos=13, Mpos=10, Rpos=18
Lpos=13, Mpos=10, Rpos=19
Lpos=13, Mpos=10, Rpos=20
Lpos=13, Mpos=10, Rpos=21
Lpos=13, Mpos=10, Rpos=22
Lpos=13, Mpos=10, Rpos=23
Lpos=13, Mpos=10, Rpos=24
Lpos=13, Mpos=10, Rpos=25
--- middle rotor step ---
Lpos=13, Mpos=11, Rpos=0
Lpos=13, Mpos=11, Rpos=1
Lpos=13, Mpos=11, Rpos=2
Lpos=13, Mpos=11, Rpos=3
Lpos=13, Mpos=11, Rpos=4
Lpos=13, Mpos=11, Rpos=5
Lpos=13, Mpos=11, Rpos=6
Lpos=13, Mpos=11, Rpos=7
Lpos=13, Mpos=11, Rpos=8
Lpos=13, Mpos=11, Rpos=9
Lpos=13, Mpos=11, Rpos=10
Lpos=13, Mpos=11, Rpos=11
Lpos=13, Mpos=11, Rpos=12
Lpos=

...

Lpos=17, Mpos=17, Rpos=3
Lpos=17, Mpos=17, Rpos=4
Lpos=17, Mpos=17, Rpos=5
Lpos=17, Mpos=17, Rpos=6
Lpos=17, Mpos=17, Rpos=7
Lpos=17, Mpos=17, Rpos=8
Lpos=17, Mpos=17, Rpos=9
Lpos=17, Mpos=17, Rpos=10
Lpos=17, Mpos=17, Rpos=11
Lpos=17, Mpos=17, Rpos=12
Lpos=17, Mpos=17, Rpos=13
Lpos=17, Mpos=17, Rpos=14
Lpos=17, Mpos=17, Rpos=15
Lpos=17, Mpos=17, Rpos=16
Lpos=17, Mpos=17, Rpos=17
Lpos=17, Mpos=17, Rpos=18
Lpos=17, Mpos=17, Rpos=19
Lpos=17, Mpos=17, Rpos=20
Lpos=17, Mpos=17, Rpos=21
Lpos=17, Mpos=17, Rpos=22
Lpos=17, Mpos=17, Rpos=23
Lpos=17, Mpos=17, Rpos=24
Lpos=17, Mpos=17, Rpos=25
--- middle rotor step ---
Lpos=17, Mpos=18, Rpos=0
Lpos=17, Mpos=18, Rpos=1
Lpos=17, Mpos=18, Rpos=2
Lpos=17, Mpos=18, Rpos=3
Lpos=17, Mpos=18, Rpos=4
Lpos=17, Mpos=18, Rpos=5
Lpos=17, Mpos=18, Rpos=6
Lpos=17, Mpos=18, Rpos=7
Lpos=17, Mpos=18, Rpos=8
Lpos=17, Mpos=18, Rpos=9
Lpos=17, Mpos=18, Rpos=10
Lpos=17, Mpos=18, Rpos=11
Lpos=17, Mpos=18, Rpos=12
Lpos=17, Mpos=18, Rpos=13
Lpos=17, Mpos=18, Rpos=14
Lpo

Lpos=19, Mpos=0, Rpos=9
Lpos=19, Mpos=0, Rpos=10
Lpos=19, Mpos=0, Rpos=11
Lpos=19, Mpos=0, Rpos=12
Lpos=19, Mpos=0, Rpos=13
Lpos=19, Mpos=0, Rpos=14
Lpos=19, Mpos=0, Rpos=15
Lpos=19, Mpos=0, Rpos=16
Lpos=19, Mpos=0, Rpos=17
Lpos=19, Mpos=0, Rpos=18
Lpos=19, Mpos=0, Rpos=19
Lpos=19, Mpos=0, Rpos=20
Lpos=19, Mpos=0, Rpos=21
Lpos=19, Mpos=0, Rpos=22
Lpos=19, Mpos=0, Rpos=23
Lpos=19, Mpos=0, Rpos=24
Lpos=19, Mpos=0, Rpos=25
--- middle rotor step ---
Lpos=19, Mpos=1, Rpos=0
Lpos=19, Mpos=1, Rpos=1
Lpos=19, Mpos=1, Rpos=2
Lpos=19, Mpos=1, Rpos=3
Lpos=19, Mpos=1, Rpos=4
Lpos=19, Mpos=1, Rpos=5
Lpos=19, Mpos=1, Rpos=6
Lpos=19, Mpos=1, Rpos=7
Lpos=19, Mpos=1, Rpos=8
Lpos=19, Mpos=1, Rpos=9
Lpos=19, Mpos=1, Rpos=10
Lpos=19, Mpos=1, Rpos=11
Lpos=19, Mpos=1, Rpos=12
Lpos=19, Mpos=1, Rpos=13
Lpos=19, Mpos=1, Rpos=14
Lpos=19, Mpos=1, Rpos=15
Lpos=19, Mpos=1, Rpos=16
Lpos=19, Mpos=1, Rpos=17
Lpos=19, Mpos=1, Rpos=18
Lpos=19, Mpos=1, Rpos=19
Lpos=19, Mpos=1, Rpos=20
Lpos=19, Mpos=1, Rpos=21
Lpos=19, M

Lpos=21, Mpos=24, Rpos=5
Lpos=21, Mpos=24, Rpos=6
Lpos=21, Mpos=24, Rpos=7
Lpos=21, Mpos=24, Rpos=8
Lpos=21, Mpos=24, Rpos=9
Lpos=21, Mpos=24, Rpos=10
Lpos=21, Mpos=24, Rpos=11
Lpos=21, Mpos=24, Rpos=12
Lpos=21, Mpos=24, Rpos=13
Lpos=21, Mpos=24, Rpos=14
Lpos=21, Mpos=24, Rpos=15
Lpos=21, Mpos=24, Rpos=16
Lpos=21, Mpos=24, Rpos=17
Lpos=21, Mpos=24, Rpos=18
Lpos=21, Mpos=24, Rpos=19
Lpos=21, Mpos=24, Rpos=20
Lpos=21, Mpos=24, Rpos=21
Lpos=21, Mpos=24, Rpos=22
Lpos=21, Mpos=24, Rpos=23
Lpos=21, Mpos=24, Rpos=24
Lpos=21, Mpos=24, Rpos=25
--- middle rotor step ---
Lpos=21, Mpos=25, Rpos=0
Lpos=21, Mpos=25, Rpos=1
Lpos=21, Mpos=25, Rpos=2
Lpos=21, Mpos=25, Rpos=3
Lpos=21, Mpos=25, Rpos=4
Lpos=21, Mpos=25, Rpos=5
Lpos=21, Mpos=25, Rpos=6
Lpos=21, Mpos=25, Rpos=7
Lpos=21, Mpos=25, Rpos=8
Lpos=21, Mpos=25, Rpos=9
Lpos=21, Mpos=25, Rpos=10
Lpos=21, Mpos=25, Rpos=11
Lpos=21, Mpos=25, Rpos=12
Lpos=21, Mpos=25, Rpos=13
Lpos=21, Mpos=25, Rpos=14
Lpos=21, Mpos=25, Rpos=15
Lpos=21, Mpos=25, Rpos=16
L

Lpos=23, Mpos=25, Rpos=14
Lpos=23, Mpos=25, Rpos=15
Lpos=23, Mpos=25, Rpos=16
Lpos=23, Mpos=25, Rpos=17
Lpos=23, Mpos=25, Rpos=18
Lpos=23, Mpos=25, Rpos=19
Lpos=23, Mpos=25, Rpos=20
Lpos=23, Mpos=25, Rpos=21
Lpos=23, Mpos=25, Rpos=22
Lpos=23, Mpos=25, Rpos=23
Lpos=23, Mpos=25, Rpos=24
Lpos=23, Mpos=25, Rpos=25
--- middle rotor step ---
Lpos=23, Mpos=0, Rpos=0
Lpos=23, Mpos=0, Rpos=1
Lpos=23, Mpos=0, Rpos=2
Lpos=23, Mpos=0, Rpos=3
Lpos=23, Mpos=0, Rpos=4
Lpos=23, Mpos=0, Rpos=5
Lpos=23, Mpos=0, Rpos=6
Lpos=23, Mpos=0, Rpos=7
Lpos=23, Mpos=0, Rpos=8
Lpos=23, Mpos=0, Rpos=9
Lpos=23, Mpos=0, Rpos=10
Lpos=23, Mpos=0, Rpos=11
Lpos=23, Mpos=0, Rpos=12
Lpos=23, Mpos=0, Rpos=13
Lpos=23, Mpos=0, Rpos=14
Lpos=23, Mpos=0, Rpos=15
Lpos=23, Mpos=0, Rpos=16
Lpos=23, Mpos=0, Rpos=17
Lpos=23, Mpos=0, Rpos=18
Lpos=23, Mpos=0, Rpos=19
Lpos=23, Mpos=0, Rpos=20
Lpos=23, Mpos=0, Rpos=21
Lpos=23, Mpos=0, Rpos=22
Lpos=23, Mpos=0, Rpos=23
Lpos=23, Mpos=0, Rpos=24
Lpos=23, Mpos=0, Rpos=25
--- middle rotor step 

Lpos=25, Mpos=23, Rpos=20
Lpos=25, Mpos=23, Rpos=21
Lpos=25, Mpos=23, Rpos=22
Lpos=25, Mpos=23, Rpos=23
Lpos=25, Mpos=23, Rpos=24
Lpos=25, Mpos=23, Rpos=25
--- middle rotor step ---
Lpos=25, Mpos=24, Rpos=0
Lpos=25, Mpos=24, Rpos=1
Lpos=25, Mpos=24, Rpos=2
Lpos=25, Mpos=24, Rpos=3
Lpos=25, Mpos=24, Rpos=4
Lpos=25, Mpos=24, Rpos=5
Lpos=25, Mpos=24, Rpos=6
Lpos=25, Mpos=24, Rpos=7
Lpos=25, Mpos=24, Rpos=8
Lpos=25, Mpos=24, Rpos=9
Lpos=25, Mpos=24, Rpos=10
Lpos=25, Mpos=24, Rpos=11
Lpos=25, Mpos=24, Rpos=12
Lpos=25, Mpos=24, Rpos=13
Lpos=25, Mpos=24, Rpos=14
Lpos=25, Mpos=24, Rpos=15
Lpos=25, Mpos=24, Rpos=16
Lpos=25, Mpos=24, Rpos=17
Lpos=25, Mpos=24, Rpos=18
Lpos=25, Mpos=24, Rpos=19
Lpos=25, Mpos=24, Rpos=20
Lpos=25, Mpos=24, Rpos=21
Lpos=25, Mpos=24, Rpos=22
Lpos=25, Mpos=24, Rpos=23
Lpos=25, Mpos=24, Rpos=24
Lpos=25, Mpos=24, Rpos=25
--- middle rotor step ---
Lpos=25, Mpos=25, Rpos=0
Lpos=25, Mpos=25, Rpos=1
Lpos=25, Mpos=25, Rpos=2
Lpos=25, Mpos=25, Rpos=3
Lpos=25, Mpos=25, Rpos=4
L

In [10]:
word = 'oh what a beautiful morning oh what a beautiful day i have a wonderful feeling everthings going my way'

In [11]:
def only_ascii(instring):
    """strips out anything that's not an ascii character (i.e a-z alphabet character), capitalises"""
    newstring = ''
    for character in instring:
        if character in ascii_letters:
            newstring += character.upper()
    return newstring

In [12]:
only_ascii(word)

'OHWHATABEAUTIFULMORNINGOHWHATABEAUTIFULDAYIHAVEAWONDERFULFEELINGEVERTHINGSGOINGMYWAY'

In [13]:
with open('menu_transfer.pickle', 'rb') as infile:
    menu = dill.load(infile)
menu

{4: {'in': 'A',
  'out': 'F',
  'menu_link': 'ZZD',
  'conx_in': {6: 'out', 12: 'in', 17: 'in', 9: 'in'},
  'conx_out': {13: 'in'}},
 13: {'in': 'F',
  'out': 'D',
  'menu_link': 'ZZM',
  'conx_in': {4: 'out'},
  'conx_out': {16: 'in'}},
 16: {'in': 'D',
  'out': 'C',
  'menu_link': 'ZZP',
  'conx_in': {13: 'out'},
  'conx_out': {6: 'in'}},
 6: {'in': 'C',
  'out': 'A',
  'menu_link': 'ZZF',
  'conx_in': {16: 'out'},
  'conx_out': {4: 'in', 12: 'in', 17: 'in', 9: 'in'}},
 12: {'in': 'A',
  'out': 'I',
  'menu_link': 'ZZL',
  'conx_in': {4: 'in', 6: 'out', 17: 'in', 9: 'in'},
  'conx_out': {}},
 17: {'in': 'A',
  'out': 'O',
  'menu_link': 'ZZQ',
  'conx_in': {4: 'in', 6: 'out', 12: 'in', 9: 'in'},
  'conx_out': {0: 'in'}},
 0: {'in': 'O',
  'out': 'Z',
  'menu_link': 'ZZZ',
  'conx_in': {17: 'out'},
  'conx_out': {}},
 9: {'in': 'A',
  'out': 'T',
  'menu_link': 'ZZI',
  'conx_in': {4: 'in', 6: 'out', 12: 'in', 17: 'in'},
  'conx_out': {11: 'in'}},
 11: {'in': 'T',
  'out': 'J',
  'men

In [14]:
class Bombe:
    
    def __init__(self,left_rotor='I',middle_rotor='II',right_rotor='III',reflector='B'):
        self.left_rotor = left_rotor
        self.middle_rotor = middle_rotor
        self.right_rotor = right_rotor
        self.reflector = reflector
        self.scramblers = {}
        
    def setup(self,menu):
        for key,val in menu.items():
            self.scramblers[key] = ScramblerBombe(self.left_rotor,self.middle_rotor,self.right_rotor,self.reflector,
                                                  menu_link=val['menu_link'],conx_in=val['conx_in'], conx_out=val['conx_out'])
    
    def pulse_connections(self):
        """Goes through every scrambler (once only), and updates in and out terminals with live feeds via conx_in and
        conx_out from its connected scramblers. Should be run iteratively to exhaustion."""
        for order,scr in self.scramblers.items():
#             print(order,scr)
            for pos,side in scr.conx_in.items():
                if side == 'in':
                    for ch,io in self.scramblers[pos].in_status.items():
                        if io == 1:
#                             print(ch)
                            scr.in_status[ch] == 1
                elif side == 'out':
                    for ch,io in self.scramblers[pos].out_status.items():
                        if io == 1:
#                             print(ch)
                            scr.in_status[ch] == 1
            for pos,side in scr.conx_out.items():
                if side == 'in':
                    for ch,io in self.scramblers[pos].in_status.items():
                        if io == 1:
#                             print(ch)
                            scr.out_status[ch] == 1
                elif side == 'out':
                    for ch,io in self.scramblers[pos].out_status.items():
                        if io == 1:
#                             print(ch)
                            scr.out_status[ch] == 1
                
    def light_character(self,in_character):
        """just need a way to put in the initial character input into all of the scramblers"""
        for scr in self.scramblers.values():
            scr.in_status[in_character] = 1
            
    def update_all(self):
        """For every scrambler, runs update() which passes live terminals through the scrambler - from in to out
        and vice versa, for whatever the current position is"""
        for scr in self.scramblers.keys():
            self.scramblers[scr].update()
            
    def spin_scramblers(self):
        """Runs step_enigma for all scramblers, spinning the right rotor once and perhaps the middle and left if
        they are in their notch positions"""
        for scr in self.scramblers.values():
            scr.step_enigma()

In [15]:
tb1 = Bombe('I','II','III','B')

In [16]:
tb1.setup(menu)

In [17]:
tb1.scramblers[9].conx_in

{4: 'in', 6: 'out', 12: 'in', 17: 'in'}

In [18]:
# tb1.scramblers[9].in_status

In [19]:
tb1.light_character('A')

In [20]:
tb1.update_all()
tb1.pulse_connections()

4 <__main__.ScramblerBombe object at 0x108319b70>
Q
A
A
A
A
13 <__main__.ScramblerBombe object at 0x108319a20>
Y
A
16 <__main__.ScramblerBombe object at 0x108319908>
Y
A
6 <__main__.ScramblerBombe object at 0x1083194a8>
F
A
A
A
A
12 <__main__.ScramblerBombe object at 0x108319a58>
A
Q
A
A
17 <__main__.ScramblerBombe object at 0x108319be0>
A
Q
A
A
A
0 <__main__.ScramblerBombe object at 0x108319c88>
Z
9 <__main__.ScramblerBombe object at 0x108319ef0>
A
Q
A
A
A
11 <__main__.ScramblerBombe object at 0x108319780>
W
A
A
1 <__main__.ScramblerBombe object at 0x108319b00>
B
A
10 <__main__.ScramblerBombe object at 0x108319d68>
B
A
A
14 <__main__.ScramblerBombe object at 0x108319080>
M


In [22]:
for scr in tb1.scramblers.values():
    print([k for k,v in scr.in_status.items() if v == 1], [k for k,v in scr.out_status.items() if v == 1])

['A'] ['Y']
['A'] ['Y']
['A'] ['F']
['A'] ['Q']
['A'] ['O']
['A'] ['Z']
['A'] ['X']
['A'] ['W']
['A'] ['B']
['A'] ['E']
['A'] ['M']
['A'] ['V']


In [31]:
tb1.spin_scramblers()
tb1.update_all()
tb1.pulse_connections()

Lpos=25, Mpos=0, Rpos=6
Lpos=25, Mpos=25, Rpos=15
Lpos=25, Mpos=25, Rpos=18
Lpos=25, Mpos=25, Rpos=8
Lpos=25, Mpos=25, Rpos=14
Lpos=25, Mpos=25, Rpos=19
Lpos=25, Mpos=25, Rpos=2
Lpos=25, Mpos=25, Rpos=11
Lpos=25, Mpos=25, Rpos=13
Lpos=25, Mpos=25, Rpos=3
Lpos=25, Mpos=25, Rpos=12
Lpos=25, Mpos=25, Rpos=16
4 <__main__.ScramblerBombe object at 0x108319b70>
C
N
Q
U
V
W
X
A
C
F
K
M
R
Z
A
F
J
L
P
R
S
W
A
D
E
K
Q
U
X
A
L
M
P
S
Z
13 <__main__.ScramblerBombe object at 0x108319a20>
G
L
O
P
R
S
X
Y
A
F
J
O
P
S
X
Z
16 <__main__.ScramblerBombe object at 0x108319908>
F
J
O
U
V
Y
A
F
H
I
O
P
T
6 <__main__.ScramblerBombe object at 0x1083194a8>
B
C
F
H
K
Q
T
Z
A
G
J
L
O
P
X
Y
A
C
F
K
M
R
Z
A
F
J
L
P
R
S
W
A
D
E
K
Q
U
X
12 <__main__.ScramblerBombe object at 0x108319a58>
A
G
J
L
O
P
X
Y
C
N
Q
U
V
W
X
A
F
J
L
P
R
S
W
A
D
E
K
Q
U
X
17 <__main__.ScramblerBombe object at 0x108319be0>
A
G
J
L
O
P
X
Y
C
N
Q
U
V
W
X
A
C
F
K
M
R
Z
A
D
E
K
Q
U
X
A
O
Q
S
U
Y
Z
0 <__main__.ScramblerBombe object at 0x108319c88>
B
C

In [32]:
for scr in tb1.scramblers.values():
    print([k for k,v in scr.in_status.items() if v == 1], [k for k,v in scr.out_status.items() if v == 1])

['A', 'G', 'J', 'L', 'O', 'P', 'X', 'Y'] ['G', 'L', 'O', 'P', 'R', 'S', 'X', 'Y']
['A', 'L', 'M', 'P', 'S', 'Z'] ['F', 'J', 'O', 'U', 'V', 'Y']
['A', 'F', 'J', 'O', 'P', 'S', 'X', 'Z'] ['B', 'C', 'F', 'H', 'K', 'Q', 'T', 'Z']
['A', 'F', 'H', 'I', 'O', 'P', 'T'] ['C', 'N', 'Q', 'U', 'V', 'W', 'X']
['A', 'C', 'F', 'K', 'M', 'R', 'Z'] ['D', 'G', 'H', 'O', 'U', 'V', 'Y']
['A', 'F', 'J', 'L', 'P', 'R', 'S', 'W'] ['B', 'C', 'H', 'O', 'Q', 'T', 'U', 'Z']
['A', 'O', 'Q', 'S', 'U', 'Y', 'Z'] ['B', 'C', 'D', 'E', 'T', 'W', 'X']
['A', 'D', 'E', 'K', 'Q', 'U', 'X'] ['B', 'G', 'L', 'M', 'O', 'P', 'W']
['A', 'C', 'F', 'G', 'Q', 'R', 'Z'] ['B', 'H', 'O', 'S', 'U', 'V', 'Y']
['A', 'C', 'E', 'O', 'Q', 'U', 'V', 'Z'] ['C', 'E', 'G', 'J', 'L', 'T', 'W', 'Y']
['A', 'D', 'I', 'J', 'Q', 'R', 'X', 'Z'] ['B', 'G', 'H', 'L', 'M', 'O', 'U', 'Y']
['A', 'B', 'E', 'L', 'R', 'S', 'X', 'Z'] ['A', 'F', 'J', 'K', 'U', 'V', 'W', 'Z']


In [33]:
for scr in tb1.scramblers.values():
    print(scr.full_scramble('A'))

S
F
C
W
U
T
T
O
V
Y
Y
Z
